In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

In [4]:
data = pd.read_csv('../../CBlog2/predicting_heart_failure/heart_failure_clinical_records_dataset.csv')

X = data.iloc[:,:-1]
y = data.iloc[:,-1]

#Loop to seperate X data into categorical and numerical groups
#The categorical features are booleans, thus the script classifies any column\
#with equa or less than 2 unique values as a categorical
categoricals_columns = []
numericals_columns = []
for i in list(X.columns):
    if data[i].nunique() > 2:
        numericals_columns.append(i)
    else:
        categoricals_columns.append(i)

categoricals = X[categoricals_columns]
numericals = X[numericals_columns]

ss = StandardScaler()
preprocessor = ColumnTransformer(
    transformers=[
        ('numericals',
        ss,
        numericals_columns
        )
    ]
)
numericals_scaled = pd.DataFrame(preprocessor.fit_transform(X), columns=numericals_columns)
X_processed = pd.concat([numericals_scaled,categoricals], axis=1)

In [5]:
X_processed.head()

,age,creatinine_phosphokinase,ejection_fraction,platelets,serum_creatinine,serum_sodium,time,anaemia,diabetes,high_blood_pressure,sex,smoking
0,1.192945,0.000166,-1.530560,1.681648e-02,0.490057,-1.504036,-1.629502,0,0,1,1,0
1,-0.491279,7.514640,-0.007077,7.535660e-09,-0.284552,-0.141976,-1.603691,0,0,0,1,0
2,0.350833,-0.449939,-1.530560,-1.038073e+00,-0.090900,-1.731046,-1.590785,0,0,0,1,1
3,-0.912335,-0.486071,-1.530560,-5.464741e-01,0.490057,0.085034,-1.590785,1,0,0,1,0
4,0.350833,-0.435486,-1.530560,6.517986e-01,1.264666,-4.682176,-1.577879,1,1,0,0,0
